In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点 AI Python SDK：使用 Python 软件包、托管文本数据集和 TF 服务容器进行自定义训练

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_Custom_Training_Python_Package_Managed_Text_Dataset_Tensorflow_Serving_Container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/sdk/SDK_Custom_Training_Python_Package_Managed_Text_Dataset_Tensorflow_Serving_Container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/sdk/SDK_Custom_Training_Python_Package_Managed_Text_Dataset_Tensorflow_Serving_Container.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="顶点 AI logo">
      在顶点 AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本笔记本演示了如何使用自定义Python包训练创建自定义模型，使用Vertex AI数据集，并使用TensorFlow-Serving容器提供在线预测和批量预测。您需要提供一个存储数据集的存储桶。

注意：在测试此SDK时，您可能会因训练、预测、存储或使用其他GCP产品而产生费用。

了解有关[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)的更多信息。

### 目标

在本教程中，您将学习如何使用自定义 Python 包训练创建自定义模型，并学习如何使用 TensorFlow-Serving 容器为在线预测提供服务。然后您将执行对模型的批量预测。

本教程使用以下 Google Cloud ML 服务和资源：

- `Vertex AI 数据集`
- `Vertex AI CustomPythonPackageTrainingJob`
- `Vertex AI 模型`资源
- `Vertex AI 端点`资源
- `Vertex AI 批处理预测`

执行的步骤包括：

- 创建实用函数以下载数据并准备 CSV 文件以创建 Vertex AI 托管数据集
- 下载数据
- 准备 CSV 文件以创建托管数据集
- 创建自定义训练 Python 包
- 创建 TensorFlow Serving 容器
- 使用托管文本数据集运行自定义 Python 包训练
- 在 Vertex AI 上部署模型并创建端点
- 在端点上进行预测
- 在模型上创建批处理预测作业

### 数据集
#### Stack Overflow数据
您可以从https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz下载并创建一个由Vertex AI管理的文本数据集。

Stack Overflow数据受知识共享署名-相同方式共享3.0国际许可协议的许可。要查看此许可协议的副本，请访问http://creativecommons.org/licenses/by-sa/3.0/。

有关此数据集的更多信息，请访问：https://console.cloud.google.com/marketplace/details/stack-exchange/stack-overflow

### 成本

本教程使用计费组件的 Google 云服务：

- Vertex AI
- 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

设置您的本地开发环境

**如果您正在使用Colab或Vertex AI Workbench笔记本**，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

**否则**，请确保您的环境满足本笔记本的要求。
您需要以下内容：

* Google Cloud SDK
* Git
* Python 3
* virtualenv
* 在使用Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南 [设置Python开发环境](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一套简要的说明：

1. [安装并初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

2. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

3. [安装
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，在终端窗口的命令行中运行 `pip3 install jupyter`。

5. 要启动Jupyter，在终端窗口的命令行中运行 `jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

安装

安装以下必需的包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform tensorflow {USER_FLAG} -q

### 重新启动内核

在安装额外的软件包后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### 设置你的Google Cloud项目

**无论你使用哪种笔记本环境，都需要完成以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当你第一次创建账户时，你将获得$300的免费信用，可以用于支付计算和存储成本。

1. [确保你的项目已启用计费功能](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

1. 如果你在本地运行这个笔记本，你需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入你的项目ID。然后运行单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分操作。以下是 Vertex AI 支持的区域。建议您选择最接近您的区域。

- 美洲: `us-central1`
- 欧洲: `europe-west4`
- 亚太: `asia-east1`

您可能不能使用多区域存储桶来进行 Vertex AI 的训练。并非所有区域都支持所有 Vertex AI 服务。

了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您在进行实时教程会话，可能会使用共享的测试账户或项目。为了避免用户在创建的资源之间发生名称冲突，您可以为每个实例会话创建一个UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

###验证您的Google Cloud账户

**如果您正在使用Vertex AI Workbench Notebooks**，您的环境已经通过验证。

**如果您正在使用Colab**，请运行下面的单元格，并按照提示进行账户oAuth验证。

**否则**，请按照以下步骤操作：

1. 在Cloud Console中，转到[**创建服务账户密钥**页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击**创建服务账户**。

3. 在**服务账户名称**字段中输入一个名称，并单击**创建**。

4. 在**授予该服务账户对项目的访问权限**部分，点击**角色**下拉列表。在过滤框中输入“Vertex AI”，并选择**Vertex AI管理员**。在过滤框中输入“存储对象管理员”，并选择**存储对象管理员**。

5. 点击*创建*。一个包含您密钥的JSON文件将下载到您的本地环境。

6. 在下面的单元格中将您的服务账户密钥路径输入为`GOOGLE_APPLICATION_CREDENTIALS`变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS '[your-service-account-key-path]'

### 创建一个云存储桶

**无论您使用的笔记本环境如何，以下步骤都是必需的。**

使用 Vertex AI SDK 提交训练作业时，您需要将包含训练代码的 Python 包上传到云存储桶中。Vertex AI 将从这个包中运行代码。在本教程中，Vertex AI 还会将训练作业产生的训练模型保存在同一个存储桶中。通过使用这个模型工件，您可以创建 Vertex AI 模型和端点资源，以便提供在线预测。

在下方设置您的云存储桶的名称。它必须在所有云存储桶中保持唯一。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有当您的存储桶尚不存在时，才执行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查内容来验证对您的云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

导入库并定义常量

In [ ]:
import csv
import os

from google.cloud import aiplatform, storage
from tensorflow.keras import utils

### 设置您的应用程序名称、任务名称和目录。

In [ ]:
APP_NAME = "keras-text-class-stack-overflow-tag"
TASK_TYPE = "mbsdk_custom-py-pkg-training"

TASK_NAME = f"{TASK_TYPE}_{APP_NAME}"

TASK_DIR = f"./{TASK_NAME}"
DATA_DIR = f"{TASK_DIR}/data"

print(f"Task Name:      {TASK_NAME}")
print(f"Task Directory: {TASK_DIR}")
print(f"Data Directory: {DATA_DIR}")

### 设置一个 GCS 前缀

如果要将所有输入和输出文件集中在 gcs 位置下。

In [ ]:
BUCKET_NAME = BUCKET_URI.split("gs://")[1]
GCS_PREFIX = f"{TASK_TYPE}/{APP_NAME}"

print(f"Bucket Name:    {BUCKET_NAME}")
print(f"GCS Prefix:     {GCS_PREFIX}")

### 用于下载数据并准备 CSV 文件以创建 Vertex AI 托管数据集的实用函数

In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    destination_file_name = os.path.join("gs://", bucket_name, destination_blob_name)

    return destination_file_name


def download_data(data_dir):
    """Download data."""

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    url = "https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"
    dataset = utils.get_file(
        "stack_overflow_16k.tar.gz",
        url,
        untar=True,
        cache_dir=data_dir,
        cache_subdir="",
    )
    data_dir = os.path.join(os.path.dirname(dataset))

    return data_dir


def upload_train_data_to_gcs(train_data_dir, bucket_name, destination_blob_prefix):
    """Create CSV file using train data content."""

    train_data_dir = os.path.join(data_dir, "train")
    train_data_fn = os.path.join(data_dir, "train.csv")

    fp = open(train_data_fn, "w", encoding="utf8")
    writer = csv.writer(
        fp, delimiter=",", quotechar='"', quoting=csv.QUOTE_ALL, lineterminator="\n"
    )

    for root, _, files in os.walk(train_data_dir):
        for file in files:
            if file.endswith(".txt"):
                class_name = root.split("/")[-1]
                file_fn = os.path.join(root, file)
                with open(file_fn) as f:
                    content = f.readlines()
                    lines = [x.strip().strip('"') for x in content]
                    writer.writerow((lines[0], class_name))

    fp.close()

    train_gcs_url = upload_blob(
        bucket_name, train_data_fn, os.path.join(destination_blob_prefix, "train.csv")
    )

    return train_gcs_url

### 下载数据

In [ ]:
data_dir = download_data(DATA_DIR)
print(f"Data is downloaded to: {data_dir}")

In [ ]:
!ls $data_dir

In [ ]:
!ls $data_dir/train

### 为创建受控数据集准备csv文件

创建包含数据内容的CSV文件

In [ ]:
gcs_source_train_url = upload_train_data_to_gcs(
    train_data_dir=os.path.join(data_dir, "train"),
    bucket_name=BUCKET_NAME,
    destination_blob_prefix=f"{GCS_PREFIX}/data",
)

print(f"Train data content is loaded to {gcs_source_train_url}")

In [ ]:
!gsutil ls gs://$BUCKET_NAME/$GCS_PREFIX/data

# 创建自定义的Python训练包

在您可以使用预先构建的容器进行自定义训练之前，您必须创建一个包含您的训练应用程序的[Python源分发包](https://docs.python.org/3/distutils/sourcedist.html)，并将其上传到您的Google Cloud项目可以访问的Cloud Storage存储桶中。

您需要创建一个目录，并将所有包构建工件写入该文件夹。

In [ ]:
PYTHON_PACKAGE_APPLICATION_DIR = f"{TASK_NAME}/trainer"

!mkdir -p $PYTHON_PACKAGE_APPLICATION_DIR
!touch $PYTHON_PACKAGE_APPLICATION_DIR/__init__.py

### 编写培训脚本

In [ ]:
%%writefile {PYTHON_PACKAGE_APPLICATION_DIR}/task.py


import os
import argparse

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import json
import tqdm

VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 250

def str2bool(v):
  if isinstance(v, bool):
    return v
  if v.lower() in ('yes', 'true', 't', 'y', '1'):
    return True
  elif v.lower() in ('no', 'false', 'f', 'n', '0'):
    return False
  else:
    raise argparse.ArgumentTypeError('Boolean value expected.')

def build_model(num_classes, loss, optimizer, metrics, vectorize_layer):
  # vocab_size is VOCAB_SIZE + 1 since 0 is used additionally for padding.
  model = tf.keras.Sequential([
      vectorize_layer,
      layers.Embedding(VOCAB_SIZE + 1, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_classes),
      layers.Activation('softmax')
  ])
  model.compile(
      loss=loss,
      optimizer=optimizer,
      metrics=metrics)

  return model

def get_string_labels(predicted_scores_batch, class_names):
  predicted_labels = tf.argmax(predicted_scores_batch, axis=1)
  predicted_labels = tf.gather(class_names, predicted_labels)
  return predicted_labels

def predict(export_model, class_names, inputs):
  predicted_scores = export_model.predict(inputs)
  predicted_labels = get_string_labels(predicted_scores, class_names)
  return predicted_labels

def parse_args():
  parser = argparse.ArgumentParser(
      description='Keras Text Classification on Stack Overflow Questions')
  parser.add_argument(
      '--epochs', default=25, type=int, help='number of training epochs')
  parser.add_argument(
      '--batch-size', default=16, type=int, help='mini-batch size')
  parser.add_argument(
      '--model-dir', default=os.getenv('AIP_MODEL_DIR'), type=str, help='model directory')
  parser.add_argument(
      '--data-dir', default='./data', type=str, help='data directory')
  parser.add_argument(
      '--test-run', default=False, type=str2bool, help='test run the training application, i.e. 1 epoch for training using sample dataset')
  parser.add_argument(
      '--model-version', default=1, type=int, help='model version')
  args = parser.parse_args()
  return args

def load_aip_dataset(aip_data_uri_pattern, batch_size, class_names, test_run, shuffle=True, seed=42):

  data_file_urls = list()
  labels = list()

  class_indices = dict(zip(class_names, range(len(class_names))))
  num_classes = len(class_names)

  for aip_data_uri in tqdm.tqdm(tf.io.gfile.glob(pattern=aip_data_uri_pattern)):
    with tf.io.gfile.GFile(name=aip_data_uri, mode='r') as gfile:
      for line in gfile.readlines():
        line = json.loads(line)
        data_file_urls.append(line['textContent'])
        classification_annotation = line['classificationAnnotations'][0]
        label = classification_annotation['displayName']
        labels.append(class_indices[label])
        if test_run:
          break

  data = list()
  for data_file_url in tqdm.tqdm(data_file_urls):
    with tf.io.gfile.GFile(name=data_file_url, mode='r') as gf:
      txt = gf.read()
      data.append(txt)

  print(f' data files count: {len(data_file_urls)}')
  print(f' data count: {len(data)}')
  print(f' labels count: {len(labels)}')

  dataset = tf.data.Dataset.from_tensor_slices(data)
  label_ds = tf.data.Dataset.from_tensor_slices(labels)
  label_ds = label_ds.map(lambda x: tf.one_hot(x, num_classes))

  dataset = tf.data.Dataset.zip((dataset, label_ds))

  if shuffle:
    # Shuffle locally at each iteration
    dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
  dataset = dataset.batch(batch_size)
  # Users may need to reference `class_names`.
  dataset.class_names = class_names

  return dataset

def main():

  args = parse_args()

  class_names = ['csharp', 'java', 'javascript', 'python']
  class_indices = dict(zip(class_names, range(len(class_names))))
  num_classes = len(class_names)
  print(f' class names: {class_names}')
  print(f' class indices: {class_indices}')
  print(f' num classes: {num_classes}')

  epochs = 1 if args.test_run else args.epochs

  aip_model_dir = os.environ.get('AIP_MODEL_DIR')
  aip_data_format = os.environ.get('AIP_DATA_FORMAT')
  aip_training_data_uri = os.environ.get('AIP_TRAINING_DATA_URI')
  aip_validation_data_uri = os.environ.get('AIP_VALIDATION_DATA_URI')
  aip_test_data_uri = os.environ.get('AIP_TEST_DATA_URI')

  print(f"aip_model_dir: {aip_model_dir}")
  print(f"aip_data_format: {aip_data_format}")
  print(f"aip_training_data_uri: {aip_training_data_uri}")
  print(f"aip_validation_data_uri: {aip_validation_data_uri}")
  print(f"aip_test_data_uri: {aip_test_data_uri}")

  print('Loading AIP dataset')
  train_ds = load_aip_dataset(
      aip_training_data_uri, args.batch_size, class_names, args.test_run)
  print('AIP training dataset is loaded')
  val_ds = load_aip_dataset(
      aip_validation_data_uri, 1, class_names, args.test_run)
  print('AIP validation dataset is loaded')
  test_ds = load_aip_dataset(
      aip_test_data_uri, 1, class_names, args.test_run)
  print('AIP test dataset is loaded')

  vectorize_layer = TextVectorization(
      max_tokens=VOCAB_SIZE,
      output_mode='int',
      output_sequence_length=MAX_SEQUENCE_LENGTH)

  train_text = train_ds.map(lambda text, labels: text)
  vectorize_layer.adapt(train_text)
  print('The vectorize_layer is adapted')


  print('Build model')
  optimizer = 'adam'
  metrics = ['accuracy']

  model = build_model(
      num_classes, losses.CategoricalCrossentropy(from_logits=True), optimizer, metrics, vectorize_layer)

  history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
  history = history.history

  print('Training accuracy: {acc}, loss: {loss}'.format(
      acc=history['accuracy'][-1], loss=history['loss'][-1]))
  print('Validation accuracy: {acc}, loss: {loss}'.format(
      acc=history['val_accuracy'][-1], loss=history['val_loss'][-1]))

  loss, accuracy = model.evaluate(test_ds)
  print('Test accuracy: {acc}, loss: {loss}'.format(
      acc=accuracy, loss=loss))

  inputs = [
      "how do I extract keys from a dict into a list?",  # python
      "debug public static void main(string[] args) {...}",  # java
  ]
  predicted_labels = predict(model, class_names, inputs)
  for input, label in zip(inputs, predicted_labels):
    print(f'Question: {input}')
    print(f'Predicted label: {label.numpy()}')

  model_export_path = os.path.join(args.model_dir, str(args.model_version))
  model.save(model_export_path)
  print(f'Model version {args.model_version} is exported to {args.model_dir}')

  loaded = tf.saved_model.load(model_export_path)
  input_name = list(loaded.signatures['serving_default'].structured_input_signature[1].keys())[0]
  print(f'Serving function input: {input_name}')

  return

if __name__ == '__main__':
  main()


### 构建包

In [ ]:
%%writefile {TASK_DIR}/setup.py

from setuptools import find_packages
from setuptools import setup

setup(
    name='trainer',
    version='0.1',
    packages=find_packages(),
    install_requires=(),
    include_package_data=True,
    description='My training application.'
)

In [ ]:
!ls $TASK_DIR

In [ ]:
!cd $TASK_DIR && python3 setup.py sdist --formats=gztar

In [ ]:
!ls -ltr $TASK_DIR/dist/trainer-0.1.tar.gz

将包上传到GCS

In [ ]:
destination_blob_name = f"custom-training-python-package/{APP_NAME}/trainer-0.1.tar.gz"
source_file_name = f"{TASK_DIR}/dist/trainer-0.1.tar.gz"

python_package_gcs_uri = upload_blob(
    BUCKET_NAME, source_file_name, destination_blob_name
)
python_module_name = "trainer.task"

print(f"Custom Training Python Package is uploaded to: {python_package_gcs_uri}")

# 创建TensorFlow Serving容器

In [ ]:
TF_SERVING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT_ID}/tf-serving"

下载 TensorFlow Serving Docker 镜像。

In [ ]:
if not IS_COLAB:
    !docker pull tensorflow/serving:2.8.0
else:
    # install docker daemon
    ! apt-get -qq install docker.io

使用容器注册表配置Docker身份验证。

In [ ]:
! gcloud auth configure-docker gcr.io --quiet

创建一个用于注册图像的标签，并使用Cloud Container Registry (gcr.io) 注册图像。

In [ ]:
if not IS_COLAB:
    !docker tag tensorflow/serving:2.8.0 $TF_SERVING_CONTAINER_IMAGE_URI
    !docker push $TF_SERVING_CONTAINER_IMAGE_URI

In [ ]:
%%bash -s $IS_COLAB $TF_SERVING_CONTAINER_IMAGE_URI
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker pull tensorflow/serving:2.8.0
docker tag tensorflow/serving:2.8.0 $2
docker push $2
kill $(jobs -p)

# 使用托管文本数据集运行自定义Python软件包培训

初始化用于Python的Vertex AI SDK

为Vertex AI SDK初始化*client*。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

创建一个Vertex AI数据集资源
您可以使用之前准备的csv文件创建一个Vertex AI文本数据集。请选择以下其中一种选项。

In [ ]:
dataset_display_name = f"temp-{APP_NAME}-content"
gcs_source = gcs_source_train_url

#### 使用csv文件创建数据集

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name=dataset_display_name,
    gcs_source=gcs_source,
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
)

## 在 Vertex AI 上启动培训任务并创建模型

接下来，您可以使用您刚刚构建的Python包来训练一个模型。

###配置一个培训任务

In [ ]:
MODEL_NAME = APP_NAME
PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-8:latest"
)

您需要指定构建并上传到GCS的Python软件包，Python软件包的模块名称，用于训练的预先构建的训练容器镜像URI，在本示例中，用于预测使用TensorFlow serving容器。

In [ ]:
job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=f"temp_{TASK_NAME}_tf-serving",
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
    model_serving_container_image_uri=TF_SERVING_CONTAINER_IMAGE_URI,
    model_serving_container_command=["/usr/bin/tensorflow_model_server"],
    model_serving_container_args=[
        f"--model_name={MODEL_NAME}",
        "--model_base_path=$(AIP_STORAGE_URI)",
        "--rest_api_port=8080",
        "--port=8500",
        "--file_system_poll_wait_seconds=31540000",
    ],
    model_serving_container_predict_route=f"/v1/models/{MODEL_NAME}:predict",
    model_serving_container_health_route=f"/v1/models/{MODEL_NAME}",
)

运行训练任务

In [ ]:
model = job.run(
    dataset=dataset,
    annotation_schema_uri=aiplatform.schema.dataset.annotation.text.classification,
    args=["--epochs", "50"],
    replica_count=1,
    model_display_name=f"temp_{TASK_NAME}_tf-serving",
    sync=False,
)

In [ ]:
model.wait()

在Vertex AI上部署模型并创建端点

部署您的模型，然后等到模型完成部署后再进行预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4", sync=False)

In [ ]:
endpoint.wait()

## 在端点上进行预测

In [ ]:
class_names = ["csharp", "java", "javascript", "python"]

class_ids = range(len(class_names))

class_indices = dict(zip(class_names, class_ids))
class_maps = dict(zip(class_ids, class_names))
print(f"Class Indices: {class_indices}")
print(f"Class Maps:    {class_maps}")

In [ ]:
text_inputs = [
    "how do I extract keys from a dict into a list?",  # python
    "debug public static void main(string[] args) {...}",  # java
]

In [ ]:
import numpy as np

predictions = endpoint.predict(instances=text_inputs)
for text, predicted_scores in zip(text_inputs, predictions.predictions):
    class_id = np.argmax(predicted_scores)
    class_name = class_maps[class_id]
    print(f"Question: {text}")
    print(f"Predicted Tag: {class_name}\n")

在模型上进行批量预测任务

In [ ]:
import json

import tensorflow as tf


def upload_test_data_to_gcs(test_data_dir, test_gcs_url):
    """Create JSON file using test data content."""

    input_name = "text_vectorization_input"

    with tf.io.gfile.GFile(test_gcs_url, "w") as gf:

        for root, _, files in os.walk(test_data_dir):
            for file in files:
                if file.endswith(".txt"):
                    file_fn = os.path.join(root, file)
                    with open(file_fn) as f:
                        content = f.readlines()
                        lines = [x.strip().strip('"') for x in content]

                        data = {input_name: lines[0]}
                        gf.write(json.dumps(data))
                        gf.write("\n")
    return

In [ ]:
gcs_source_test_url = f"gs://{BUCKET_NAME}/{GCS_PREFIX}/data/test.json"
upload_test_data_to_gcs(
    test_data_dir=os.path.join(data_dir, "test"), test_gcs_url=gcs_source_test_url
)

print(f"Test data content is loaded to {gcs_source_test_url}")

In [ ]:
!gsutil ls $gcs_source_test_url

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=f"temp_{TASK_NAME}_tf-serving",
    gcs_source=gcs_source_test_url,
    gcs_destination_prefix=f"gs://{BUCKET_NAME}/{GCS_PREFIX}/batch_prediction",
    machine_type="n1-standard-4",
    sync=False,
)

In [ ]:
batch_predict_job.wait()
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_errors_stats = list()
prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction.errors_stats"):
        prediction_errors_stats.append(blob.name)
    if blob.name.split("/")[-1].startswith("prediction.results"):
        prediction_results.append(blob.name)

In [ ]:
tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            text = line["instance"]["text_vectorization_input"][0]
            prediction = line["prediction"]
            class_id = np.argmax(prediction)
            class_name = class_maps[class_id]
            tags.append([text, class_name])

In [ ]:
import pandas as pd

tags_df = pd.DataFrame(tags, columns=["question", "tag"])
tags_df.head()

In [ ]:
tags_df["tag"].value_counts()

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Undeploy model from the endpoint
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI